In [1]:
from __future__ import division
from numpy.random import randn
from pandas import Series
import numpy as np
import pandas as pd
from datetime import datetime
import math
np.set_printoptions(precision=4)
import sys; sys.path.append('book_scripts')
%pwd

u'E:\\project\\pychram\\trade_scripts'

In [2]:
def load_stock_base_msg():
    stock_data_path = 'E:/project/pychram/traderesp/base/output-csv/'
    stock_data_cvs = stock_data_path + 'output-2013-2014-all-stock-base-msg.csv'
    #
    stat = pd.read_csv(stock_data_cvs,encoding='gbk',index_col=['code'])
    return stat

In [3]:
def get_date_period_staticstics(begtime,endtime):
    #
    stock_data_path = 'E:/project/pychram/traderesp/strategy/output-csv/'
    stock_data_cvs = stock_data_path + 'output-stock-history-adjust-m26-kdj-jx-buy-sx-sell-' \
                                     + begtime.strftime('%Y%m%d') + '-' + endtime.strftime('%Y%m%d') + '.csv'
    #
    stat = pd.read_csv(stock_data_cvs,parse_dates=['buy_date','sell_date'],encoding='gbk')
    
    #逐笔利润
    stat['profit'] = stat['sell_price'] - stat['buy_price']
    #
    action_num = len(stat)
    suc_num = stat[stat['profit'] > 0.00 ].count() 
    fail_num = action_num - suc_num
    #
    tmp_profit = stat.groupby(['code']).sum()
    year_action_num = len(tmp_profit)
    tmp_profit['profit_rate'] = (tmp_profit['sell_price'] - tmp_profit['buy_price']) / tmp_profit['buy_price']
    year_suc_num = tmp_profit[tmp_profit['profit_rate'] > 0.00].count()
    year_fail_num = year_action_num - year_suc_num
    #
    year_suc_num_gt_percet_5 = tmp_profit[tmp_profit['profit_rate'] > 0.05].count()
    #
    stat_sum_all = stat.sum()
    code_unique_num = len(stat['code'].unique())
    s_ge_b_num = stat[stat['sell_price'] > stat['buy_price']].count()
    #
    print '-----  ',begtime,'  ----  ',endtime,'  -----'
    print '股票总数(排重) :',year_action_num
    print '买卖总次数(一次买卖算一次) :',action_num
    print '其中获利次数 :',suc_num['profit']
    print '其中亏损次数 :',fail_num['profit']
    print '获利次数比率 :',suc_num['profit'] / action_num
    print '亏损次数比率 :',fail_num['profit'] / action_num
    print '年度获利股票数 :',year_suc_num['profit_rate']
    print '年度亏损股票数 :',year_fail_num['profit_rate']
    print '年度获利股票数比率 :',year_suc_num['profit_rate'] / year_action_num
    print '年度亏损股票数比率 :',year_fail_num['profit_rate'] / year_action_num
    print '年度获利股票数( 盈利大于5%) :',year_suc_num_gt_percet_5['profit_rate']
    print '年度获利股票数比率( 盈利大于5%) :',year_suc_num_gt_percet_5['profit_rate'] / year_action_num
    print '---------------------------------------------------------------------'
    #
    return tmp_profit[tmp_profit['profit_rate'] > 0.00].sort_values('profit_rate',ascending=False),tmp_profit,stat

In [4]:
def show_date_staticstics():
    code1,code1_tmp1,code1_tmp2 = get_date_period_staticstics(datetime(2013,1,1),datetime(2013,12,31))
    code2,code2_tmp1,code2_tmp2 = get_date_period_staticstics(datetime(2014,1,1),datetime(2014,12,31))
    code3 = set(code1.index) & set(code2.index)
    year_2_num = len(code3)
    print '连续2年上涨股票数(获利股票数) :',year_2_num
    print '连续2年上涨股票数比率 :',year_2_num / len(code1) if len(code1) > 0 else 0
    #--------------------------------------------------------
    code1_gt_percent_5 =  code1[code1['profit_rate'] > 0.05]
    code2_gt_percent_5 =  code2[code2['profit_rate'] > 0.05]
    code3_gt_percent_5 =  set(code1_gt_percent_5.index) & set(code2_gt_percent_5.index)
    print '连续2年上涨股票数(获利股票数 大于 5%) :',len(code3_gt_percent_5)
    print '连续2年上涨股票数比率(获利大于 5%) :',len(code3_gt_percent_5) / len(code1) if len(code1) > 0 else 0
    #--------------------------------------------------------
    code4 = code1.index[0:100]
    code5 = code2.index[0:100]
    code6 = set(code4) & set(code5)
    year_2_num_from_100 = len(code6)
    print '连续2年上涨股票数(Top 100) :',year_2_num_from_100
    code7 = code1.index[0:200]
    code8 = code2.index[0:200]
    code9 = set(code7) & set(code8)
    year_2_num_from_200 = len(code9)
    print '连续2年上涨股票数(Top 200) :',year_2_num_from_200
    code10 = code1.index[0:300]
    code11 = code2.index[0:300]
    code12 = set(code10) & set(code11)
    year_2_num_from_300 = len(code12)
    print '连续2年上涨股票数(Top 300) :',year_2_num_from_300
    code13 = code1.index[0:400]
    code14 = code2.index[0:400]
    code15 = set(code13) & set(code14)
    year_2_num_from_400 = len(code15)
    print '连续2年上涨股票数(Top 400) :',year_2_num_from_400
    code16 = code1.index[0:500]
    code17 = code2.index[0:500]
    code18 = set(code16) & set(code17)
    year_2_num_from_500 = len(code18)
    print '连续2年上涨股票数(Top 500) :',year_2_num_from_500
    #
    code_bing = pd.merge(code1,code2,left_index=True, right_index=True)
    return code_bing,code_bing[(code_bing['profit_rate_x'] > 0.05) & (code_bing['profit_rate_y'] > 0.05)]

In [5]:
code_base_msg = load_stock_base_msg()
code_profit_rate_ge_0, code_profit_rate_ge_5 = show_date_staticstics()
code_profit_rate_ge_5 = code_profit_rate_ge_5[['profit_rate_x','profit_rate_y']]

-----   2013-01-01 00:00:00   ----   2013-12-31 00:00:00   -----
股票总数(排重) : 2466
买卖总次数(一次买卖算一次) : 21995
其中获利次数 : 9314
其中亏损次数 : 12681
获利次数比率 : 0.423459877245
亏损次数比率 : 0.576540122755
年度获利股票数 : 1206
年度亏损股票数 : 1260
年度获利股票数比率 : 0.489051094891
年度亏损股票数比率 : 0.510948905109
年度获利股票数( 盈利大于5%) : 56
年度获利股票数比率( 盈利大于5%) : 0.0227088402271
---------------------------------------------------------------------
-----   2014-01-01 00:00:00   ----   2014-12-31 00:00:00   -----
股票总数(排重) : 2572
买卖总次数(一次买卖算一次) : 31498
其中获利次数 : 14321
其中亏损次数 : 17177
获利次数比率 : 0.454663788177
亏损次数比率 : 0.545336211823
年度获利股票数 : 1407
年度亏损股票数 : 1165
年度获利股票数比率 : 0.547045101089
年度亏损股票数比率 : 0.452954898911
年度获利股票数( 盈利大于5%) : 80
年度获利股票数比率( 盈利大于5%) : 0.0311041990669
---------------------------------------------------------------------
连续2年上涨股票数(获利股票数) : 661
连续2年上涨股票数比率 : 0.548092868988
连续2年上涨股票数(获利股票数 大于 5%) : 1
连续2年上涨股票数比率(获利大于 5%) : 0.000829187396352
连续2年上涨股票数(Top 100) : 3
连续2年上涨股票数(Top 200) : 11
连续2年上涨股票数(Top 300) : 27
连续2年上涨股票数(Top 400) :

In [6]:
code_bing = pd.merge(code_base_msg,code_profit_rate_ge_5,left_index=True, right_index=True)

In [7]:
code_bing.columns

Index([    u'code_name',       u'sina_hy',       u'sina_gn',       u'sina_dy',
       u'profit_rate_x', u'profit_rate_y'],
      dtype='object')

In [8]:
def trim_float_num(x):
    #print type(x)
    #print 'x=',x
    xx = 0.000
    if x is None:
        xx = 0.000
    else:
        xx = float(str(x))
    # print xx
    if xx < 0.001:
        return "0.1 %"
    else:
        ret = '%.1f' %(xx * 100.00)
        return ret + ' %'

In [9]:
trim_profit_rate_x = code_bing['profit_rate_x'].map(trim_float_num)
code_bing['profit_rate_x'] = trim_profit_rate_x
trim_profit_rate_y = code_bing['profit_rate_y'].map(trim_float_num)
code_bing['profit_rate_y'] = trim_profit_rate_y

In [10]:
code_bing

,code_name,sina_hy,sina_gn,sina_dy,profit_rate_x,profit_rate_y
code,,,,,,
300254,仟源医药,生物制药；生物制药,基因概念；基因概念,山西,12.0 %,7.1 %


In [11]:
code1,code_2,code_all = get_date_period_staticstics(datetime(2013,1,1),datetime(2013,12,31))

-----   2013-01-01 00:00:00   ----   2013-12-31 00:00:00   -----
股票总数(排重) : 2466
买卖总次数(一次买卖算一次) : 21995
其中获利次数 : 9314
其中亏损次数 : 12681
获利次数比率 : 0.423459877245
亏损次数比率 : 0.576540122755
年度获利股票数 : 1206
年度亏损股票数 : 1260
年度获利股票数比率 : 0.489051094891
年度亏损股票数比率 : 0.510948905109
年度获利股票数( 盈利大于5%) : 56
年度获利股票数比率( 盈利大于5%) : 0.0227088402271
---------------------------------------------------------------------


In [12]:
code_all[code_all['code']==858]

,buy_date,buy_money,buy_num,buy_price,code,sell_date,sell_money,sell_num,sell_price,profit
1884,2013-05-07,34363.793698,100,343.637937,858,2013-05-16,34408.866445,100,344.088664,0.450727
3647,2013-05-20,35280.362187,100,352.803622,858,2013-05-28,36467.380855,100,364.673809,11.870187
4408,2013-05-29,36467.380855,100,364.673809,858,2013-05-31,35686.026710,100,356.860267,-7.813541
9316,2013-08-02,29965.847459,100,299.658475,858,2013-08-20,30650.775421,100,306.507754,6.849280
13450,2013-09-10,29670.072415,100,296.700724,858,2013-09-18,28720.498028,100,287.204980,-9.495744
15068,2013-10-09,28035.572552,100,280.355726,858,2013-10-11,27833.200574,100,278.332006,-2.023720
16668,2013-10-15,28611.534744,100,286.115347,858,2013-10-18,28066.689769,100,280.666898,-5.448450
18889,2013-12-02,25420.349756,100,254.203498,858,2013-12-03,25902.904256,100,259.029043,4.825545
20102,2013-12-04,26074.148356,100,260.741484,858,2013-12-12,25544.863745,100,255.448637,-5.292846


In [13]:
code2,code21,code2_all = get_date_period_staticstics(datetime(2014,1,1),datetime(2014,12,31))

-----   2014-01-01 00:00:00   ----   2014-12-31 00:00:00   -----
股票总数(排重) : 2572
买卖总次数(一次买卖算一次) : 31498
其中获利次数 : 14321
其中亏损次数 : 17177
获利次数比率 : 0.454663788177
亏损次数比率 : 0.545336211823
年度获利股票数 : 1407
年度亏损股票数 : 1165
年度获利股票数比率 : 0.547045101089
年度亏损股票数比率 : 0.452954898911
年度获利股票数( 盈利大于5%) : 80
年度获利股票数比率( 盈利大于5%) : 0.0311041990669
---------------------------------------------------------------------


In [14]:
code2_all[code2_all['code']==858]

,buy_date,buy_money,buy_num,buy_price,code,sell_date,sell_money,sell_num,sell_price,profit
1763,2014-02-13,23381.133728,100,233.811337,858,2014-02-25,23521.240673,100,235.212407,1.401069
4063,2014-04-03,26759.129144,100,267.591291,858,2014-04-17,27506.330482,100,275.063305,7.472013
7271,2014-06-06,26385.559649,100,263.855596,858,2014-06-18,26728.031458,100,267.280315,3.424718
9346,2014-07-09,28054.832917,100,280.548329,858,2014-07-10,28719.339690,100,287.193397,6.645068
15375,2014-08-14,31247.645423,100,312.476454,858,2014-08-25,30583.155550,100,305.831556,-6.644899
17172,2014-09-03,31490.769961,100,314.907700,858,2014-09-12,30810.059593,100,308.100596,-6.807104
20920,2014-09-25,29934.818811,100,299.348188,858,2014-10-14,29075.766834,100,290.757668,-8.590520
24356,2014-11-11,29010.926904,100,290.109269,858,2014-11-19,28362.656321,100,283.626563,-6.482706
27452,2014-11-27,29270.226438,100,292.702264,858,2014-12-10,33338.259738,100,333.382597,40.680333
30812,2014-12-23,35218.302154,100,352.183022,858,2014-12-31,34845.539180,100,348.455392,-3.727630


In [15]:
code2_all[code2_all['code']==858].sum()

buy_money     290753.345128
buy_num         1000.000000
buy_price       2907.533451
code            8580.000000
sell_money    293490.379519
sell_num        1000.000000
sell_price      2934.903795
profit            27.370344
dtype: float64

In [16]:
code21.ix[858]

buy_money      290753.345128
buy_num          1000.000000
buy_price        2907.533451
sell_money     293490.379519
sell_num         1000.000000
sell_price       2934.903795
profit             27.370344
profit_rate         0.009414
Name: 858, dtype: float64

In [17]:
code_all[code_all['code']==600985]

,buy_date,buy_money,buy_num,buy_price,code,sell_date,sell_money,sell_num,sell_price,profit
3606,2013-05-20,2480.871882,100,24.808719,600985,2013-05-27,2487.125662,100,24.871257,0.062538
5096,2013-05-28,2510.056960,100,25.100570,600985,2013-06-03,2449.599157,100,24.495992,-0.604578
8222,2013-08-02,1974.263627,100,19.742636,600985,2013-08-15,2032.637314,100,20.326373,0.583737
12389,2013-08-23,2104.163759,100,21.041638,600985,2013-09-12,2327.429569,100,23.274296,2.232658
13333,2013-09-16,2390.618272,100,23.906183,600985,2013-09-17,2335.856380,100,23.358564,-0.547619
17415,2013-10-09,2382.191550,100,23.821915,600985,2013-10-18,2548.585630,100,25.485856,1.663941
19513,2013-11-21,2466.444618,100,24.664446,600985,2013-12-03,2643.371320,100,26.433713,1.769267
21887,2013-12-04,2708.665235,100,27.086652,600985,2013-12-17,2664.433874,100,26.644339,-0.442314


In [18]:
code_all[code_all['code']==600985].sum()

buy_money       19017.275904
buy_num           800.000000
buy_price         190.172759
code          4807880.000000
sell_money      19489.038905
sell_num          800.000000
sell_price        194.890389
profit              4.717630
dtype: float64

In [19]:
code2_all[code2_all['code']==600985]

,buy_date,buy_money,buy_num,buy_price,code,sell_date,sell_money,sell_num,sell_price,profit
1396,2014-02-12,2533.845834,100,25.338458,600985,2014-02-21,2519.104282,100,25.191043,-0.147416
3474,2014-04-03,2367.451694,100,23.674517,600985,2014-04-14,2329.538929,100,23.295389,-0.379128
5691,2014-04-18,2350.601463,100,23.506015,600985,2014-04-22,2274.775308,100,22.747753,-0.758262
12863,2014-07-29,2550.698905,100,25.506989,600985,2014-08-08,2538.063559,100,25.380636,-0.126353
14892,2014-08-12,2586.508264,100,25.865083,600985,2014-08-21,2744.477908,100,27.444779,1.579696
18992,2014-09-03,2787.059863,100,27.870599,600985,2014-09-17,2927.156353,100,29.271564,1.400965
20808,2014-09-24,3075.743338,100,30.757433,600985,2014-10-13,3496.033039,100,34.960330,4.202897
25194,2014-11-13,3468.440080,100,34.684401,600985,2014-12-01,3625.514073,100,36.255141,1.570740
29997,2014-12-12,3689.195381,100,36.891954,600985,2014-12-24,3808.064767,100,38.080648,1.188694


In [20]:
code2_all[code2_all['code']==600985].sum()

buy_money       25409.544822
buy_num           900.000000
buy_price         254.095448
code          5408865.000000
sell_money      26262.728219
sell_num          900.000000
sell_price        262.627282
profit              8.531834
dtype: float64